In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")

import warnings
from warnings import filterwarnings

filterwarnings("ignore")
import re
import string
!pip install contractions
import contractions
import string


from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Model
from keras.layers import Embedding, Input
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download("wordnet2022")
! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet 







In [ ]:
df = pd.read_json(
    "/kaggle/input/cell-phones-and-accessories-5/Cell_Phones_and_Accessories_5.json",
    lines=True,
)

print(len(df))

In [ ]:
SetSize = 10_000
pos_df = df[df["overall"] > 3].head(SetSize + 400)[["reviewText", "overall"]]
neg_df = df[df["overall"] < 3].head(SetSize + 600)[["reviewText", "overall"]]

In [ ]:
pos_df["overall"] = 1
neg_df["overall"] = 0


df = (
    pd.concat([pos_df, neg_df], axis=0, ignore_index=True)
    .sample(frac=1)
    .reset_index(drop=True)
)
print(len(pos_df), len(neg_df))

## Loading and Filtering Columns


In [ ]:
df = df[["reviewText", "overall"]]
print(len(df))

In [ ]:
def defineSentiment(rating):
    if rating == 1:
        return "Positive"
    else:
        return "Negative"


def defineTextLen(text):
    if isinstance(text, str):
        return len(text)
    else:
        return 0


df.loc[:, "sentiment"] = df["overall"].apply(defineSentiment)

df.loc[:, "textLen"] = df["reviewText"].apply(defineTextLen)

In [ ]:
df = df[df["textLen"] > 2]
df.reset_index(drop=True, inplace=True)

print(df.describe())

In [ ]:
px = (
    df["sentiment"]
    .value_counts()
    .sort_index()
    .plot(
        kind="pie",
        title="Sentiment Pie Distribution",
        autopct="%1.1f%%",
        ylabel="",
    )
)

plt.show()

In [ ]:
ax = df["overall"].value_counts().sort_index().plot(kind="bar", title="Review Scores")
ax.set_xlabel("Review Stars")
plt.show()

### Removing Number, Fullstop and turning Uppercase to lowercase


## Applying Contraction


In [ ]:
def contract(text):
    if isinstance(text, str):
        return contractions.fix(text)
    else:
        return "Bad String"


df["reviewText"] = df["reviewText"].apply(contract)


df_new = df.copy()

In [ ]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = "".join(
        [char for char in text if char not in string.punctuation]
    )  # Remove punctuation
    words = word_tokenize(text)  # Tokenize
    words = [
        lemmatizer.lemmatize(word) for word in words if word not in stop_words
    ]  # Lemmatization and stopword removal
    return " ".join(words)


df_new["reviewText"] = df_new["reviewText"].apply(preprocess_text)

In [ ]:
def tokenize(text):
    tokens = Tokenizer()
    tokens.fit_on_texts(text)
    return tokens.texts_to_sequences(text), tokens.word_index


corpus = df_new["reviewText"].tolist()


df_new["seq"], vocab_size = tokenize(corpus)

print(df_new["seq"])

In [ ]:
seqlen = [len(seq) for seq in df_new["seq"]]
plt.plot(seqlen)
plt.xlabel("Index")
plt.ylabel("Length")
plt.show()
pd.Series(seqlen).describe()

In [ ]:
maxlen = int(np.percentile(seqlen, 90)) + 1
print(maxlen)

In [ ]:
df_new = df_new[df_new["seq"].apply(len) <= maxlen]
len(df_new)

In [ ]:
sequences = df_new["seq"].tolist()

padded_X = pad_sequences(
    sequences,
    maxlen=maxlen,
    dtype="int32",
    padding="post",
    truncating="post",
    value=0.0,
)

len(padded_X)

In [ ]:
len(df_new["overall"])

In [ ]:
y = df_new["overall"]


y = np.array(y)
split_index = (len(y) * 0.8).__ceil__()

y_train = y[:split_index]
y_test = y[split_index:]

X_train = padded_X[:split_index]
X_test = padded_X[split_index:]

print(f"Training Size: {len(X_train)}")
print(f"Test Size: X: {len(X_test)} y: {len(y_test)}")

In [ ]:
print("Vocab Size ", len(vocab_size))
print("Max Len ", maxlen)

In [ ]:
model = Sequential()
model.add(
    Embedding(input_dim=(len(vocab_size) + 1), output_dim=300, input_length=maxlen)
)
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Bidirectional(LSTM(256)))
model.add(Dropout(0.5))
model.add(Dense(2, activation="softmax"))
optimizer = Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss", patience=2, factor=0.5, min_lr=1e-6
)
model.summary()

In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=20,
    validation_split=0.2,
    callbacks=[lr_scheduler],
)

In [ ]:
pred = model.predict(X_test)
pred

In [ ]:
# pd.DataFrame({"pred":pred,"y_test":y_test})
y_pred = np.argmax(pred, axis=1)
print(y_pred)

In [ ]:
hits = np.sum(y_pred == y_test)
misses = np.sum(y_pred != y_test)

print(f"Total Samples: {len(y_test)}")
print(f"Hits (Correct Predictions): {hits}")
print(f"Misses (Incorrect Predictions): {misses}")
print(f"Accuracy: {hits / len(y_test) * 100:.2f}%")

In [ ]:
y_acc = y_test[y_pred != y_test]
pd.Series(y_acc).value_counts()

In [ ]:
# Save model weights to a file
model.save("sentiment_model.h5")